In [1]:
# Function to run test cases
def run_tests(func, test_cases):
    if not test_cases:
        raise ValueError("No test cases provided.")
    
    if not isinstance(test_cases, list):
        raise TypeError("Test cases should be provided as a list.")
    
    for i, test_case in enumerate(test_cases):
        if not isinstance(test_case, list):
            raise TypeError(f"Test case {i+1} is not a list.")
        
        # Handle both single arguments and lists of arguments
        if not test_case:
            raise ValueError(f"Test case {i+1} is empty.")
        
        if isinstance(test_case[0], list):
            result = func(test_case)
            test_case_str = str(test_case)
        else:
            result = func(*test_case)
            test_case_str = str(test_case)
        
        print(f"Case {i+1}: {test_case_str} -> {result}")

# Azure AI Language

## Overview
Uses the [Azure AI Language](https://learn.microsoft.com/en-us/azure/ai-services/language-service/) API to do text analysis. You will need an API key to use these functions.

## Sentiment Analysis

Arguments:

| Argument      | Positional | Type           | Description                                                                 |
|---------------|------------|----------------|-----------------------------------------------------------------------------|
| `documents`   | arg1       | string or list | The text documents to analyze. Can be a single string or a pandas DataFrame.|
| `azureai_key` | arg2       | string         | The Azure AI API key. e.g. 7920c673894d49d292de11a80d16572                  |
| `azureai_url` | arg3       | string         | The Azure AI API endpoint (e.g. https://boardflare-free.cognitiveservices.azure.com/)       |

Returns a list of lists containing the sentiment analysis results. Each inner list contains:

| Return Value     | Type  | Description                                                                                  |
|------------------|-------|----------------------------------------------------------------------------------------------|
| DocIndex         | int   | Document index (1-based).                                                                    |
| DSent            | string| Document sentiment.                                                                          |
| DPos             | float | Document confidence score (positive).                                                        |
| DNeu             | float | Document confidence score (neutral).                                                         |
| DNeg             | float | Document confidence score (negative).                                                        |
| Sentence         | string| Sentence text.                                                                               |
| SSent            | string| Sentence sentiment.                                                                          |
| SPos             | float | Sentence confidence score (positive).                                                        |
| SNeu             | float | Sentence confidence score (neutral).                                                         |
| SNeg             | float | Sentence confidence score (negative).                                                       |
| Aspect           | string| Target text.                                                                                 |
| ASent            | string| Target sentiment.                                                                            |
| APos             | float | Target confidence score (positive).                                                          |
| ANeg             | float | Target confidence score (negative).                                                          |
| Opinion          | string| Assessment text.                                                                             |
| OSent            | string| Assessment sentiment.                                                                        |
| OPos             | float | Assessment confidence score (positive).                                                      |
| ONeg             | float | Assessment confidence score (negative).                                                      |

Sentiment labels (e.g. positive) are returned at the document, sentence, and aspect level, with a confidence score for each.  The labels are positive, negative, and neutral. At the document level, the mixed sentiment label also can be returned. The sentiment of the document is determined below:

| Sentence sentiment | Returned document label |
|--------------------|-------------------------|
| At least one `positive` sentence is in the document. The rest of the sentences are `neutral`. | `positive` |
| At least one `negative` sentence is in the document. The rest of the sentences are `neutral`. | `negative` |
| At least one `negative` sentence and at least one `positive` sentence are in the document. | `mixed` |
| All sentences in the document are `neutral`. | `neutral` |

Confidence scores range from 1 to 0. Scores closer to 1 indicate a higher confidence in the label's classification, while lower scores indicate lower confidence. For each document or each sentence, the predicted scores associated with the labels (positive, negative, and neutral) add up to 1. For more information, see the Responsible AI transparency note.

Each aspect and the corresponding opinion are extracted, also known as Aspect-based Sentiment Analysis, which provides more granular information about the opinions related to attributes of products or services in text. The API surfaces opinions as a target (noun or verb) and an assessment (adjective).

For example, if a customer leaves feedback about a hotel such as "The room was great, but the staff was unfriendly.", the following results will be returned as outlined in the table below:

| DocIndex | DSent   | DPos | DNeu | DNeg | Sentence                                      | SSent    | SPos | SNeu | SNeg | Aspect | ASent    | APos | ANeg | Opinion    | OSent    | OPos | ONeg |
|----------|---------|------|------|------|-----------------------------------------------|----------|------|------|------|--------|----------|------|------|------------|----------|------|------|
| 1        | negative| 0    | 0    | 1    | The room was great, but the staff was unfriendly. | negative | 0    | 0    | 1    | room   | positive | 1    | 0    | great      | positive | 1    | 0    |
| 1        | negative| 0    | 0    | 1    | The room was great, but the staff was unfriendly. | negative | 0    | 0    | 1    | staff  | negative | 0    | 1    | unfriendly | negative | 0    | 1    |

Notes:
- Document has "negative" sentiment
- First sentence about the room is positive with high confidence
- Second sentence about staff is negative with high confidence
- Aspect-level analysis correctly identifies "room" and "staff" as targets
- Associated opinions "great" and "unfriendly" are correctly matched to their aspects


In [2]:
import micropip
await micropip.install(["azure-ai-textanalytics"])
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import pandas as pd

# Example method for detecting sentiment and opinions in text 
def azure_sentiment(documents, language_key, language_endpoint):
    """
    Analyzes sentiment and opinions in the provided documents using Azure Text Analytics.

    Parameters:
    documents (str or DataFrame): The text documents to analyze. Can be a single string or a pandas DataFrame.
    language_key (str): The Azure Text Analytics API key.
    language_endpoint (str): The Azure Text Analytics API endpoint.

    Returns:
    list: A list of lists containing the sentiment analysis results. Each inner list contains:
        - Document index (1-based)
        - Document sentiment
        - Document confidence scores (positive, neutral, negative)
        - Sentence text
        - Sentence sentiment
        - Sentence confidence scores (positive, neutral, negative)
        - Target text
        - Target sentiment
        - Target confidence scores (positive, negative)
        - Assessment text
        - Assessment sentiment
        - Assessment confidence scores (positive, negative)
    """
    # Check if documents is a string, list, or DataFrame
    if isinstance(documents, str):
        documents = [documents]
    elif isinstance(documents, pd.DataFrame):
        documents = documents.values.flatten().tolist()  # Convert df to list

    # Authenticate the client using your key and endpoint 
    ta_credential = AzureKeyCredential(language_key)
    client = TextAnalyticsClient(
            endpoint=language_endpoint, 
            credential=ta_credential)

    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    data = []

    for idx, document in enumerate(doc_result):
        for sentence in document.sentences:
            for mined_opinion in sentence.mined_opinions:
                target = mined_opinion.target
                for assessment in mined_opinion.assessments:
                    data.append([
                        idx + 1,  # 1-based index
                        document.sentiment,
                        document.confidence_scores.positive,
                        document.confidence_scores.neutral,
                        document.confidence_scores.negative,
                        sentence.text,
                        sentence.sentiment,
                        sentence.confidence_scores.positive,
                        sentence.confidence_scores.neutral,
                        sentence.confidence_scores.negative,
                        target.text,
                        target.sentiment,
                        target.confidence_scores.positive,
                        target.confidence_scores.negative,
                        assessment.text,
                        assessment.sentiment,
                        assessment.confidence_scores.positive,
                        assessment.confidence_scores.negative
                    ])
    return data

# Define variables for API key and endpoint
# Note this is a free tier endpoint for testing purposes which may not work if usage has exceeeded the limit.
# Please sign up for your own Azure Language free tier account for extended testing so others can try out this endpoint.
api_key = "22s0r9q9VpH9YrJjCpd9SbKw8vJS4jAJCNZ03jW8AXmKbTI1lroKJQQJ99AKACYeBjFXJ3w3AAAaACOGVdOH"
endpoint = "https://boardflare-free.cognitiveservices.azure.com/"

test_cases = [
    ["The hotel stay was fantastic overall! The room was spacious and had a beautiful view, but was dirty. The staff were incredibly attentive, but some were not very friendly. The restaurant served delicious food with a wide variety of options, and the service was top-notch. Overall, it was a wonderful experience, and I would highly recommend this hotel and restaurant to anyone looking for a great stay.", api_key, endpoint]
]

run_tests(azure_sentiment, test_cases)

# Excel usage: =AZURE_SENTIMENT("The room was great, but the staff was unfriendly.", "22s0r9q9VpH9YrJjCpd9SbKw8vJS4jAJCNZ03jW8AXmKbTI1lroKJQQJ99AKACYeBjFXJ3w3AAAaACOGVdOH", "https://boardflare-free.cognitiveservices.azure.com/")

Case 1: ['The hotel stay was fantastic overall! The room was spacious and had a beautiful view, but was dirty. The staff were incredibly attentive, but some were not very friendly. The restaurant served delicious food with a wide variety of options, and the service was top-notch. Overall, it was a wonderful experience, and I would highly recommend this hotel and restaurant to anyone looking for a great stay.', '22s0r9q9VpH9YrJjCpd9SbKw8vJS4jAJCNZ03jW8AXmKbTI1lroKJQQJ99AKACYeBjFXJ3w3AAAaACOGVdOH', 'https://boardflare-free.cognitiveservices.azure.com/'] -> [[1, 'mixed', 0.6, 0.0, 0.4, 'The hotel stay was fantastic overall! ', 'positive', 1.0, 0.0, 0.0, 'hotel stay', 'positive', 1.0, 0.0, 'fantastic', 'positive', 1.0, 0.0], [1, 'mixed', 0.6, 0.0, 0.4, 'The room was spacious and had a beautiful view, but was dirty. ', 'negative', 0.0, 0.0, 1.0, 'room', 'positive', 0.67, 0.33, 'spacious', 'positive', 1.0, 0.0], [1, 'mixed', 0.6, 0.0, 0.4, 'The room was spacious and had a beautiful view, but

In [3]:
def azure_abstractive_summary(text, language_key, language_endpoint):
    """Generate an abstractive summary of the given text using Azure AI Language.
    
    Args:
        text (str or list): Text document(s) to summarize. Can be a single string or list of strings.
        language_key (str): The Azure Language API key
        language_endpoint (str): The Azure Language API endpoint URL
        
    Returns:
        list: List of generated summary texts
    """
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import TextAnalyticsClient
    
    # Convert single string to list
    if isinstance(text, str):
        text = [text]
        
    client = TextAnalyticsClient(
        endpoint=language_endpoint,
        credential=AzureKeyCredential(language_key)
    )

    poller = client.begin_abstract_summary(text)
    results = poller.result()
    
    summaries = []
    for result in results:
        if result.kind == "AbstractiveSummarization":
            summaries.extend([summary.text for summary in result.summaries])
        elif result.is_error:
            raise Exception(f"Error: {result.error.code} - {result.error.message}")
            
    return summaries

# Test cases using sample article text
test_cases = [
    ["At Microsoft, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic, human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI Cognitive Services, I have been working with a team of amazing scientists and engineers to turn this quest into a reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes of human cognition: monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the intersection of all three, there's magic-what we call XYZ-code as illustrated in Figure 1-a joint representation to create more powerful AI that can speak, hear, see, and understand humans better.",
    "22s0r9q9VpH9YrJjCpd9SbKw8vJS4jAJCNZ03jW8AXmKbTI1lroKJQQJ99AKACYeBjFXJ3w3AAAaACOGVdOH",
    "https://boardflare-free.cognitiveservices.azure.com/"]
]

# Excel usage: =AZURE_SUMMARIZE("At Microsoft, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic, human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI Cognitive Services, I have been working with a team of amazing scientists and engineers to turn this quest into a reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes of human cognition: monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the intersection of all three, there's magic-what we call XYZ-code as illustrated in Figure 1-a joint representation to create more powerful AI that can speak, hear, see, and understand humans better.", "22s0r9q9VpH9YrJjCpd9SbKw8vJS4jAJCNZ03jW8AXmKbTI1lroKJQQJ99AKACYeBjFXJ3w3AAAaACOGVdOH", "https://boardflare-free.cognitiveservices.azure.com/")

run_tests(azure_abstractive_summary, test_cases)


Case 1: ["At Microsoft, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic, human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI Cognitive Services, I have been working with a team of amazing scientists and engineers to turn this quest into a reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes of human cognition: monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the intersection of all three, there's magic-what we call XYZ-code as illustrated in Figure 1-a joint representation to create more powerful AI that can speak, hear, see, and understand humans better.", '22s0r9q9VpH9YrJjCpd9SbKw8vJS4jAJCNZ03jW8AXmKbTI1lroKJQQJ99AKACYeBjFXJ3w3AAAaACOGVdOH', 'https://boardflare-free.cognitiveservices.azure.com/'] -> ["The Chief Technology Officer of Azure AI Cognitive Services at Microsoft highlights the company'dedrive to push the bo